# 2024 Bracket Simulation
This notebook uses a previously optimized simulation to predict this year's March Madness

This optimized model included and `XGBoost` model with:
- `Features`: 'badj_em_diff', 'wab_diff', 'barthag_diff', 'talent_diff', 'elite_sos_diff', 'win_percent_diff', 'pppo_diff', 'k_off_diff'
- `Parameters`: 
    - n_estimators=300
    - max_depth=7
    - learning_rate=0.01
    - subsample=0.8
    - colsample_bytree=0.8
    - gamma=0

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

## Train the Model
Load in all of the historical data and train the model and all of it to prepare for the 2024 predictions:

In [3]:
matchups = pd.read_csv("matchups.csv")

In [4]:
predictors = ['badj_em_diff', 'wab_diff', 'barthag_diff', 'talent_diff', 'elite_sos_diff', 'win_percent_diff', 'pppo_diff', 'k_off_diff']
target = "winner"

# Train model on rest of the years
xgb_pipeline = make_pipeline(StandardScaler(), 
                             XGBClassifier(n_estimators=300,
                             max_depth=7,
                             learning_rate=0.01,
                             subsample=0.8,
                             colsample_bytree=0.8,
                             gamma=0
                             ))

xgb_pipeline.fit(matchups[predictors], matchups[target])

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.8, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.01,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=7, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

**This fitted pipeline will be used throughout the 2024 simulation.**

## Data Processing
I did not have updated 2024 data when I ran the simulation first, so I will have to perform the same preprocessing on the new data. This will just involve copy and pasting over code from the previous preprocessing notebooks. 

In [5]:
raw_matchups = pd.read_csv("Tournament Matchups 2024.csv")
raw_stats = pd.read_csv("KenPom Barttorvik 2024.csv")

raw_stats = raw_stats[(raw_stats.YEAR == 2024)]

In [6]:
raw_matchups.head()

,YEAR,BY YEAR NO,BY ROUND NO,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE
0,2024,2036,0,1067,Connecticut,1,1,64,NaN
1,2024,2035,0,1026,Stetson,16,64,64,NaN
2,2024,1972,0,1060,Florida Atlantic,8,1,64,NaN
3,2024,1971,0,1036,Northwestern,9,64,64,NaN
4,2024,2000,0,1029,San Diego St.,5,1,64,NaN


In [7]:
raw_stats.head()

,YEAR,CONF,CONF ID,QUAD NO,QUAD ID,TEAM NO,TEAM ID,TEAM,SEED,ROUND,...,BADJT RANK,AVG HGT RANK,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK
0,2024,MAC,17,61,1,1079,2,Akron,14,0,...,276,238,199,19,176,164,47,122,51,249
1,2024,SEC,28,63,3,1078,3,Alabama,4,0,...,13,33,8,156,106,10,314,2,263,7
2,2024,P12,24,63,3,1077,8,Arizona,2,0,...,16,50,37,196,7,195,134,8,14,47
3,2024,SEC,28,64,4,1076,12,Auburn,4,0,...,58,86,76,127,69,59,284,12,8,69
4,2024,B12,7,63,3,1075,14,Baylor,3,0,...,274,31,22,304,34,97,254,15,155,1


## Round of 68 redictions

My simulation only does the round of 64 and beyond, but the round of 68 games have not been played yet.

I need to predict these and alter the data set based on those results before I continue with preprocessing.

In [8]:
r68matchups = raw_matchups[raw_matchups["CURRENT ROUND"] == 68]

r68_matchup_stats = pd.merge(r68matchups, raw_stats, on=["YEAR", "TEAM"], how="left").drop(columns=["SEED_y", "ROUND_y"])
r68_matchup_stats.rename(columns={"SEED_x":"SEED", "ROUND_x":"ROUND"}, inplace=True)

Since I will be using the next step matchup processing again, it will be a function:

In [9]:
def make_matchups(teams):
    matchups = pd.DataFrame(columns=['year', 'team_1', 'seed_1', 'round_1', 'current_round', 'score_1',
                                     'team_2', 'seed_2', 'round_2', 'score_2'])

    matchup_info_list = []
    # iterate through data frame and jump 2 each iteration
    for i in range(0, len(teams), 2):
        team1_info = teams.iloc[i]
        team2_info = teams.iloc[i+1]

        matchup_info = {
            'year': team1_info['YEAR'],
            'team_1': team1_info['TEAM'],
            'seed_1': team1_info['SEED'],
            'round_1': team1_info['ROUND'],
            'score_1' : team1_info['SCORE'],
            'score_2' : team2_info['SCORE'],
            'current_round': team1_info['CURRENT ROUND'],
            'team_2': team2_info['TEAM'],
            'seed_2': team2_info['SEED'],
            'round_2': team2_info['ROUND'],
            'badj_em_1': team1_info['BADJ EM'],
            'wab_1': team1_info['WAB'],
            'barthag_1': team1_info['BARTHAG'],
            'talent_1': team1_info['TALENT'],
            'elite_sos_1': team1_info['ELITE SOS'],
            'win_percent_1': team1_info['WIN%'],
            'pppo_1': team1_info['PPPO'],
            'k_off_1': team1_info['K OFF'],
            'badj_em_2': team2_info['BADJ EM'],
            'wab_2': team2_info['WAB'],
            'barthag_2': team2_info['BARTHAG'],
            'talent_2': team2_info['TALENT'],
            'elite_sos_2': team2_info['ELITE SOS'],
            'win_percent_2': team2_info['WIN%'],
            'pppo_2': team2_info['PPPO'],
            'k_off_2': team2_info['K OFF']
        }
        matchup_info_list.append(matchup_info)

    matchups = pd.concat([matchups, pd.DataFrame(matchup_info_list)])
        
    # get the stat differences same as before
    stat_variables = ['badj_em', 'wab', 'barthag', 'talent', 'elite_sos', 'win_percent', 'pppo', 'k_off']
    for variable in stat_variables:
        matchups[f'{variable}_diff'] = matchups[f'{variable}_1'] - matchups[f'{variable}_2']
        
    return matchups

In [10]:
# Process the round of 68 matchups
r68_matchups = make_matchups(r68_matchup_stats)

In [11]:
# Make predictions for the round of 68
r68_predictions = xgb_pipeline.predict(r68_matchups[predictors])
r68_matchups["R68 Prediction"] = r68_predictions

In [12]:
r68_matchups

,year,team_1,seed_1,round_1,current_round,score_1,team_2,seed_2,round_2,score_2,...,k_off_2,badj_em_diff,wab_diff,barthag_diff,talent_diff,elite_sos_diff,win_percent_diff,pppo_diff,k_off_diff,R68 Prediction
0,2024,Howard,16,64,68,NaN,Wagner,16,64,NaN,...,97.4514,1.607,-0.4,0.051,31.116,0.340,1.724138,0.091,8.9186,1
1,2024,Boise St.,10,64,68,NaN,Colorado,10,64,NaN,...,113.5280,-2.951,-0.8,-0.032,-19.982,1.610,-3.921569,-0.029,-2.4400,1
2,2024,Montana St.,16,64,68,NaN,Grambling St.,16,64,NaN,...,97.5516,5.367,-4.6,0.138,-12.302,-5.726,-6.451613,0.080,7.4674,0
3,2024,Virginia,10,64,68,NaN,Colorado St.,10,64,NaN,...,112.1110,-3.012,-0.3,-0.031,39.424,-1.437,0.946970,-0.094,-9.3220,0


I will now just manually drop Wagner, Colorado, Montana St. and Virginia from the original df because they were predicted to lose:

In [13]:
teams_to_drop = ["Wagner", "Colorado", "Montana St.", "Virginia"]
raw_matchups = raw_matchups[~raw_matchups['TEAM'].isin(teams_to_drop)]
raw_matchups["CURRENT ROUND"] = 64

## Preprocessing Continued
Now, back to the planned preprocessing steps with only round of 64 teams.

In [14]:
## merge dfs
raw_matchup_stats = pd.merge(raw_matchups, raw_stats, on=["YEAR", "TEAM"], how="left").drop(columns=["SEED_y", "ROUND_y"])
raw_matchup_stats.rename(columns={"SEED_x":"SEED", "ROUND_x":"ROUND"}, inplace=True)

Upon inspection, I found some failed merges (NaNs that should be there), so I will go through a similar process to last time to resolve this. 

In [15]:
unique_teams_matchups = set(raw_matchups["TEAM"].unique())
unique_teams_stats = set(raw_stats["TEAM"].unique())

# Teams present in raw_matchups but not in stats_data
teams_only_in_matchups = unique_teams_matchups - unique_teams_stats

# Teams present in stats_data but not in raw_matchups
teams_only_in_stats = unique_teams_stats - unique_teams_matchups

print("Teams only in matchups:", teams_only_in_matchups)
print("Teams only in stats:", teams_only_in_stats)

Teams only in matchups: set()
Teams only in stats: {'Wagner', 'Montana St.', 'Virginia', 'Colorado'}


The teams only in stats are fine because I dropped those when I predicted them to lose in the round of 16. However, the teams only in matchups need to be resolved. 

In [16]:
## merge rows
r64_2024_matchups = make_matchups(raw_matchup_stats)

In [17]:
r64_2024_matchups.head()

,year,team_1,seed_1,round_1,current_round,score_1,team_2,seed_2,round_2,score_2,...,pppo_2,k_off_2,badj_em_diff,wab_diff,barthag_diff,talent_diff,elite_sos_diff,win_percent_diff,pppo_diff,k_off_diff
0,2024,Connecticut,1,1,64,NaN,Stetson,16,64,NaN,...,1.112,111.410,39.218,18.1,0.615,38.477,20.981,29.886148,0.118,11.298
1,2024,Florida Atlantic,8,1,64,NaN,Northwestern,9,64,NaN,...,1.122,112.160,-3.714,-0.3,-0.056,-13.964,-10.433,10.132576,0.051,5.209
2,2024,San Diego St.,5,1,64,NaN,UAB,12,64,NaN,...,1.111,111.082,11.223,5.3,0.216,21.680,12.628,2.083333,-0.025,-2.370
3,2024,Auburn,4,1,64,NaN,Yale,13,64,NaN,...,1.102,110.425,18.873,8.0,0.213,33.957,11.838,10.446247,0.072,6.939
4,2024,BYU,6,1,64,NaN,Duquesne,11,64,NaN,...,1.048,104.828,11.477,4.7,0.146,19.283,11.228,2.049911,0.121,12.098


## Simulation Functions
I used a few functions in the last notebook that operated as the simulator, so I will bring those over here now (I realize all this copy and pasting isn't the best deployment strategy, but I will work on improving that after this project is due).

Explanation of each of these functions can be found in the mm_bracket_simulator.ipynb notebook.

In [18]:
def score_bracket(predicted, actual):
    
    score = 0
    for (pred_index, pred_matchup), (act_index, act_matchup) in zip(predicted.iterrows(), actual.iterrows()):
        
        if (pred_matchup["team_1"] == act_matchup["team_1"]) and (pred_matchup["prediction"] == act_matchup["winner"] == 1):
            score += 64 / pred_matchup["current_round"]
            
        elif (pred_matchup["team_2"] == act_matchup["team_2"]) and (pred_matchup["prediction"] == act_matchup["winner"] == 0): 
            score += 64 / pred_matchup["current_round"]
            
    return score

In [19]:
def get_winner_info(matchups):
    next_round_teams_list = []
    
    for index, matchup in matchups.iterrows():
        # if team_1 wins, get all info that ends in "_1"
        if matchup["prediction"] == 1:
            winning_team_info = matchup.filter(regex='_1$').rename(lambda x: x[:-2], axis=0)
        # if team_2 wins, get all info that ends in "_2"
        else:
            winning_team_info = matchup.filter(regex='_2$').rename(lambda x: x[:-2], axis=0)
        
        winning_team_info["year"] = matchup["year"]
        winning_team_info["current_round"] = matchup["current_round"] / 2
        
        next_round_teams_list.append(pd.DataFrame(winning_team_info).T)
    
    next_round_teams = pd.concat(next_round_teams_list, ignore_index=True)
        
    return next_round_teams

In [20]:
def next_sim_matchups(winning_teams):
    matchups = pd.DataFrame(columns=['year', 'team_1', 'seed_1', 'round_1', 'current_round', 'team_2', 'seed_2', 'round_2'])

    matchup_info_list = []
    # iterate through data frame and jump 2 each iteration
    for i in range(0, len(winning_teams)-1, 2):
        team1_info = winning_teams.iloc[i]
        team2_info = winning_teams.iloc[i+1]

        matchup_info = {
            'year': team1_info['year'],
            'team_1': team1_info['team'],
            'seed_1': team1_info['seed'],
            'round_1': team1_info['round'],
            'current_round': team1_info['current_round'],
            'team_2': team2_info['team'],
            'seed_2': team2_info['seed'],
            'round_2': team2_info['round'],
            'badj_em_1': team1_info['badj_em'],
            'wab_1': team1_info['wab'],  
            'barthag_1': team1_info['barthag'],
            'talent_1': team1_info['talent'],
            'elite_sos_1': team1_info['elite_sos'],
            'win_percent_1': team1_info['win_percent'],
            'pppo_1': team1_info['pppo'],
            'k_off_1': team1_info['k_off'],
            'badj_em_2': team2_info['badj_em'],
            'wab_2': team2_info['wab'],  
            'barthag_2': team2_info['barthag'],
            'talent_2': team2_info['talent'],
            'elite_sos_2': team2_info['elite_sos'],
            'win_percent_2': team2_info['win_percent'],
            'pppo_2': team2_info['pppo'],
            'k_off_2': team2_info['k_off']
        }
        matchup_info_list.append(matchup_info)

    matchups = pd.concat([matchups, pd.DataFrame(matchup_info_list)])
        
    # get the stat differences same as before
    stat_variables = ['badj_em', 'wab', 'barthag', 'talent', 'elite_sos', 'win_percent', 'pppo', 'k_off']
    for variable in stat_variables:
        matchups[f'{variable}_diff'] = matchups[f'{variable}_1'] - matchups[f'{variable}_2']
        
    return matchups

In [21]:
def sim_bracket(round_matchups, model):

    # get predictions for each game in the current round and add that column to the df
    preds = model.predict(round_matchups[predictors])
    round_matchups.loc[:, "prediction"] = preds
    
    # add in probabilities too in case I want to identify the most likely upsets
    probs = model.predict_proba(round_matchups[predictors])
    round_matchups.loc[:, "win probability"] = probs[:, 1]

    
    # base case for recursion (we are in the championship round)
    if round_matchups["current_round"].iloc[0] == 2:
        return round_matchups
    
    # pass teams on to the next round in a new df and combine them into new matchups
    next_round_teams = get_winner_info(round_matchups)
    next_round_matchups = next_sim_matchups(next_round_teams)

    # recurse through making a simulated df that mimics the structure of the actual df
    return pd.concat([round_matchups, sim_bracket(next_round_matchups, model)], ignore_index=True)

## Simulate 2024 March Madness
Pass in the 2024 round of 64 matchups and the previously trained model, then output the resulting bracket:

In [22]:
bracket2024 = sim_bracket(r64_2024_matchups, xgb_pipeline)

In [26]:
bracket2024[bracket2024.current_round == 2]

,year,team_1,seed_1,round_1,current_round,score_1,team_2,seed_2,round_2,score_2,...,badj_em_diff,wab_diff,barthag_diff,talent_diff,elite_sos_diff,win_percent_diff,pppo_diff,k_off_diff,prediction,win probability
62,2024,Connecticut,1,1,2.0,NaN,Houston,1,1,NaN,...,-0.204,0.7,-0.008,-5.288,-0.842,2.941176,0.1,9.651,0,0.469225


In [24]:
bracket2024.to_csv("BracketPredictions2024.csv")

I will now enter this bracket into various sites (ESPN CBS, etc) and see how it does when the tournament ends in a month.